# Importing libraries

In [27]:
import torch
import torchvision
import os
import numpy as np
import argparse
import time
import pdb
import sys
import torch.multiprocessing
import json
import lmdb
import random
import pickle

from torch import nn
from tqdm import tqdm
from datetime import datetime
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from io import BytesIO
from types import SimpleNamespace
from glob import glob

# Dataset

In [4]:
class EmbeddingDataset(Dataset):
    def __init__(self, image_path, text_path, part=None):
        embeddings = self.read_pickle(image_path)
        self.image_embeddings = torch.tensor(embeddings[0])
        if part == 'all':
            self.text_embeddings = torch.tensor(embeddings[1])
        else:
            text_embeddings = self.read_pickle(text_path)
            self.text_embeddings = torch.tensor(text_embeddings[0])
            

    def __getitem__(self, idx):
        return (self.image_embeddings[idx], self.text_embeddings[idx])

    def __len__(self):
        return len(self.image_embeddings)

    def read_pickle(self, fpath):
        objects = []
        with (open(fpath, "rb")) as openfile:
            while True:
                try:
                    objects.append(pickle.load(openfile))
                except EOFError:
                    break
        if len(objects) == 1:
            return objects[0]
        return objects


# Model

In [5]:
class Embedder(nn.Module):
    def __init__(self, output_size, input_size=1024):
        super().__init__()

        self.layer1 = nn.Sequential(
            nn.Linear(input_size, 512),
            nn.BatchNorm1d(512),
            nn.Dropout(),
            nn.LeakyReLU()
        )
        self.layer2 = nn.Linear(512, output_size)

    def forward(self, x):
        x = self.layer1(x)
        return self.layer2(x)

# Loss

In [23]:
class TripletLoss:
    def __init__(self, margin=None):
        self.margin = margin
        if margin is not None:
            self.ranking_loss = nn.MarginRankingLoss(margin=margin)
        else:
            self.ranking_loss = nn.SoftMarginLoss()

    def __call__(self, dist_ap, dist_an):
        y = torch.ones_like(dist_ap)
        if self.margin is not None:
            loss = self.ranking_loss(dist_an, dist_ap, y)
        else:
            loss = self.ranking_loss(dist_an - dist_ap, y)
        return loss

def normalize(x, axis=-1):
    x = 1. * x / (torch.norm(x, 2, axis, keepdim=True).expand_as(x) + 1e-12)
    return x


def euclidean_dist(x, y):
    """
    Args:
        x: pytorch Variable, with shape [m, d]
        y: pytorch Variable, with shape [n, d]
    Returns:
        dist: pytorch Variable, with shape [m, n]
    """
    m, n = x.size(0), y.size(0)
    xx = torch.pow(x, 2).sum(1, keepdim=True).expand(m, n)
    # import pdb; pdb.set_trace()
    yy = torch.pow(y, 2).sum(1, keepdim=True).expand(n, m).t()
    dist = xx + yy
    # dist.addmm_(1, -2, x, y.t())
    dist.addmm_(x, y.t(), beta=1, alpha=-2)
    dist = dist.clamp(min=1e-12).sqrt()  # for numerical stability
    return dist


def hard_example_mining(dist_mat, labels, return_inds=False):
    assert len(dist_mat.size()) == 2
    assert dist_mat.size(0) == dist_mat.size(1)
    N = dist_mat.size(0)

    # shape [N, N]
    is_pos = labels.expand(N, N).eq(labels.expand(N, N).t())
    is_neg = labels.expand(N, N).ne(labels.expand(N, N).t())

    # `dist_ap` means distance(anchor, positive)
    # both `dist_ap` and `relative_p_inds` with shape [N, 1]
    dist_ap, relative_p_inds = torch.max(
    dist_mat[is_pos].contiguous().view(N, -1), 1, keepdim=True)
    # `dist_an` means distance(anchor, negative)
    # both `dist_an` and `relative_n_inds` with shape [N, 1]
    dist_an, relative_n_inds = torch.min(
        dist_mat[is_neg].contiguous().view(N, -1), 1, keepdim=True)

    # shape [N]
    dist_ap = dist_ap.squeeze(1)
    dist_an = dist_an.squeeze(1)

    return dist_ap, dist_an


def triplet_loss_fn(tri_loss, global_feat, labels, normalize_feature=False):

    if normalize_feature:
        global_feat = normalize(global_feat, axis=-1)
    # shape [N, N]
    dist_mat = euclidean_dist(global_feat, global_feat)
    # import pdb; pdb.set_trace()
    dist_ap, dist_an = hard_example_mining(
        dist_mat, labels, return_inds=False)
    loss = tri_loss(dist_ap, dist_an)
    return loss, dist_ap, dist_an, dist_mat

# Helper functions

In [44]:
def compute_ranks(rcps, imgs, retrieved_type='recipe'):
    assert imgs.shape == rcps.shape, 'recipe features and image features should have same dimension'
    # pdb.set_trace()
    imgs = imgs / np.linalg.norm(imgs, axis=1)[:, None]
    rcps = rcps / np.linalg.norm(rcps, axis=1)[:, None]
    if retrieved_type == 'recipe':
        sims = np.dot(imgs, rcps.T) # [N, N]
    else:
        sims = np.dot(rcps, imgs.T)
    
    ranks = []
    preds = []
    # loop through the N similarities for images
    for ii in range(imgs.shape[0]):
        # get a column of similarities for image ii
        sim = sims[ii,:]
        # sort indices in descending order
        sorting = np.argsort(sim)[::-1].tolist()
        # find where the index of the pair sample ended up in the sorting
        pos = sorting.index(ii)
        ranks.append(pos+1.0)
        preds.append(sorting[0])
    # pdb.set_trace()
    return np.asarray(ranks), preds

def pickler(file_, path, mode='ab'):
    dbfile_paths = open(path, mode) 
    # source, destination 
    pickle.dump(file_, dbfile_paths)                      
    dbfile_paths.close()

def rank(rcps, imgs, retrieved_type='recipe', retrieved_range=1000, draw_hist=False, verbose=False, epoch=-1):
    t1 = time.time()
    N = retrieved_range
    data_size = imgs.shape[0]
    glob_rank = []
    glob_recall = {1:0.0, 5:0.0, 10:0.0}
    pickler(imgs, 'image_outputs.pkl')
    pickler(rcps, 'recipe_outputs.pkl')
    if draw_hist:
        plt.figure(figsize=(16, 6))
    # average over 10 sets
    for i in range(10):
        ids_sub = np.random.choice(data_size, N, replace=False)
        imgs_sub = imgs[ids_sub, :]
        rcps_sub = rcps[ids_sub, :]
        # loop through the N similarities for images
        ranks, _ = compute_ranks(rcps_sub, imgs_sub, retrieved_type)

        pickler(ranks, 'ranks.pkl')
        pickler(ids_sub, 'indices.pkl')

        recall = {1:0.0, 5:0.0, 10:0.0}
        for ii in recall.keys():
            recall[ii] = (ranks<=ii).sum() / ranks.shape[0]
        med = int(np.median(ranks))
        for ii in recall.keys():
            glob_recall[ii] += recall[ii]
        glob_rank.append(med)
        if draw_hist:
            ranks = np.array(ranks)
            plt.subplot(2,5,i+1)
            n, bins, patches = plt.hist(x=ranks, bins='auto', color='#0504aa', alpha=0.7, rwidth=0.85)
            plt.grid(axis='y', alpha=0.75)
            plt.ylim(top=300)
            plt.text(23, 45, 'avgR(std) = {:.2f}({:.2f})\nmedR={:.2f}\n#<{:d}:{:d}|#={:d}:{:d}|#>{:d}:{:d}'.format(
                np.mean(ranks), np.std(ranks), np.median(ranks), 
                med,(ranks<med).sum(), med,(ranks==med).sum(), med,(ranks>med).sum()))
    if draw_hist:
        plt.savefig(f'hist_{epoch}.png')
    
    for i in glob_recall.keys():
        glob_recall[i] = glob_recall[i]/10

    medR = np.mean(glob_rank)
    medR_std = np.std(glob_rank)
    t2 = time.time()
    if verbose:
        print(f'=>retrieved_range={retrieved_range}, MedR={medR:.4f}({medR_std:.4f}), time={t2-t1:.4f}s')
    return medR, medR_std, glob_recall

def create_retrieval_model(args, device='cuda', image_encoder='resnet'):
    txt_encoder = Embedder(args.latent_size).to(device)
    img_encoder = Embedder(args.latent_size).to(device)
    optimizer = torch.optim.Adam(
        [
            {'params': txt_encoder.parameters()},
            {'params': img_encoder.parameters()},
        ], 
        lr=args.lr
    )
    return txt_encoder, img_encoder, optimizer

def load_retrieval_model(ckpt_path, device='cuda', image_encoder='resnet'):
    print(f'load retrieval model from {ckpt_path}')
    ckpt = torch.load(ckpt_path)
    
    ckpt_args = ckpt['args']
    if 'lr' not in ckpt_args.__dict__:
        ckpt_args.lr = 0.0001

    txt_encoder, img_encoder, optimizer = create_retrieval_model(ckpt_args, device, image_encoder)
    epoch_start = ckpt['epoch']+1
    txt_encoder.load_state_dict(ckpt['txt_encoder'])
    img_encoder.load_state_dict(ckpt['img_encoder'])
    optimizer.load_state_dict(ckpt['optimizer'])
    
    return ckpt_args, epoch_start, txt_encoder, img_encoder, optimizer

def save_retrieval_model(args, txt_module, img_module, optimizer, epoch, ckpt_path):
    print(f'save retrieval model to {ckpt_path}')
    ckpt = {
        'args': args,
        'epoch': epoch,
        'txt_encoder': txt_module.state_dict(),
        'img_encoder': img_module.state_dict(),
        'optimizer': optimizer.state_dict(),
    }
    torch.save(ckpt, ckpt_path)

def train_triplet_loss(
    args, train_loader, val_loader, 
    txt_encoder, img_encoder, optimizer, 
    n_train, n_val):
    
    save_dir = args.save_dir
    device = args.device
    if device == 'cuda':
        txt_module = txt_encoder.module
        img_module = img_encoder.module
    else:
        txt_module = txt_encoder
        img_module = img_encoder
    
    ckpt_path = os.path.join(save_dir, f'model.pt')
    triplet_loss = TripletLoss(args.margin)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=3)
    min_val_loss = float('inf')
    for epoch in range(args.epochs):
        time.sleep(0.5)
        print('#' * 40)
        print(f'Epoch = {epoch}')
        
        print('train')
        txt_encoder.train()
        img_encoder.train()
        train_loss = 0.0
        time.sleep(0.5)
        iteration = 1
        for img, txt in tqdm(train_loader):
            txt_output = txt_encoder(txt.to(device))
            img_output = img_encoder(img.to(device))

            bs = img.shape[0]
            label = list(range(0, bs))
            label.extend(label)
            label = np.array(label)
            label = torch.tensor(label).long().to(device)
            loss = triplet_loss_fn(triplet_loss, torch.cat((img_output, txt_output)), label)[0]
            train_loss += loss.item() * bs
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            iteration += 1

        train_loss /= n_train
        
        time.sleep(0.5)
        print('val')
        txt_encoder.eval()
        img_encoder.eval()
        txt_outputs = []
        img_outputs = []
        val_loss = 0.0

        time.sleep(0.5)
        with torch.no_grad():
            iteration = 0
            for txt, img in tqdm(val_loader):

                txt_output = txt_encoder(txt.to(device))
                img_output = img_encoder(img.to(device))

                txt_outputs.append(txt_output.detach().cpu())
                img_outputs.append(img_output.detach().cpu())

                bs = img.shape[0]
                label = list(range(0, bs))
                label.extend(label)
                label = np.array(label)
                label = torch.tensor(label).long().to(device)
                loss = triplet_loss_fn(triplet_loss, torch.cat((img_output, txt_output)), label)[0]
                val_loss += loss.item() * bs

                iteration += 1

        val_loss /= n_val
        txt_outputs = torch.cat(txt_outputs, dim=0).numpy()
        img_outputs = torch.cat(img_outputs, dim=0).numpy()
        retrieved_range = min(txt_outputs.shape[0], args.retrieved_range)
        medR, medR_std, recalls = rank(
            txt_outputs, img_outputs, epoch=epoch, retrieved_type=args.retrieved_type, 
            retrieved_range=retrieved_range, verbose=True)

        ckpt_path = os.path.join(save_dir, f'model.pt')
        if val_loss < min_val_loss:
            min_val_loss = val_loss
            print(f'save to {ckpt_path}')
            ckpt = {
                'args': args,
                'epoch': epoch,
                'txt_encoder': txt_module.state_dict(),
                'img_encoder': img_module.state_dict(),
                'optimizer': optimizer.state_dict(),
                'val_loss': val_loss
            }
            torch.save(ckpt, ckpt_path)

        log = {
            'epoch': epoch,
            'train_loss': train_loss,
            'val_loss': val_loss,
            'medR': medR,
            'medR_std': medR_std
        }
        for k, v in recalls.items():
            log["Recall"+str(k)] = v

        print(log)
        with open(os.path.join(save_dir, 'metrics.txt'), 'a') as f:
            f.write('\n' + str(log) + '\n')
        scheduler.step(recalls[1])

def train_mse(
    args, train_loader, val_loader, 
    txt_encoder, img_encoder, optimizer, 
    n_train, n_val):
    
    save_dir = args.save_dir
    device = args.device
    if device == 'cuda':
        txt_module = txt_encoder.module
        img_module = img_encoder.module
    else:
        txt_module = txt_encoder
        img_module = img_encoder
    
    ckpt_path = os.path.join(save_dir, f'model.pt')
    mse_loss = nn.MSELoss()
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=3)
    min_val_loss = float('inf')
    for epoch in range(args.epochs):
        time.sleep(0.5)
        print('#' * 40)
        print(f'Epoch = {epoch}')
        
        print('train')
        txt_encoder.train()
        img_encoder.train()
        train_loss = 0.0
        time.sleep(0.5)
        for img, txt in tqdm(train_loader):
            txt_output = txt_encoder(txt.to(device))
            img_output = img_encoder(img.to(device))

            loss = mse_loss(img_output, txt_output)
            train_loss += loss.item() * bs
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        train_loss /= n_train
        
        time.sleep(0.5)
        print('val')
        txt_encoder.eval()
        img_encoder.eval()
        txt_outputs = []
        img_outputs = []
        val_loss = 0.0

        time.sleep(0.5)
        with torch.no_grad():
            for txt, img in tqdm(val_loader):

                txt_output = txt_encoder(txt.to(device))
                img_output = img_encoder(img.to(device))

                txt_outputs.append(txt_output.detach().cpu())
                img_outputs.append(img_output.detach().cpu())

                loss = mse_loss(img_output, txt_output)
                val_loss += loss.item() * bs

        val_loss /= n_val
        txt_outputs = torch.cat(txt_outputs, dim=0).numpy()
        img_outputs = torch.cat(img_outputs, dim=0).numpy()
        retrieved_range = min(txt_outputs.shape[0], args.retrieved_range)
        medR, medR_std, recalls = rank(
            txt_outputs, img_outputs, epoch=epoch, retrieved_type=args.retrieved_type, 
            retrieved_range=retrieved_range, verbose=True)

        ckpt_path = os.path.join(save_dir, f'model.pt')
        if val_loss < min_val_loss:
            min_val_loss = val_loss
            print(f'save to {ckpt_path}')
            ckpt = {
                'args': args,
                'epoch': epoch,
                'txt_encoder': txt_module.state_dict(),
                'img_encoder': img_module.state_dict(),
                'optimizer': optimizer.state_dict(),
                'val_loss': val_loss
            }
            torch.save(ckpt, ckpt_path)

        log = {
            'epoch': epoch,
            'train_loss': train_loss,
            'val_loss': val_loss,
            'medR': medR,
            'medR_std': medR_std
        }
        for k, v in recalls.items():
            log["Recall"+str(k)] = v

        print(log)
        with open(os.path.join(save_dir, 'metrics.txt'), 'a') as f:
            f.write('\n' + str(log) + '\n')
        scheduler.step(recalls[1])

# Setting parameters

In [46]:
args = SimpleNamespace(
    device='cuda:7',
    seed=8,
    epochs=30,
    lr=0.0001,
    margin=0.3,
    retrieved_type='image',
    retrieved_range=1000,
    latent_size=256
)
   
torch.manual_seed(args.seed)
np.random.seed(args.seed)
torch.backends.cudnn.benchmark = True
device = args.device
image_pkl_path='/common/home/as3503/as3503/courses/cs536/dataset/prof_embeddings/embeddings_train1.pkl'
text_pkl_path='/common/home/as3503/as3503/courses/cs536/dataset/prof_embeddings/#replace#_embeddings_train.pkl'
batch_size=1024
parts = ['all', 'title', 'ingredients', 'instructions']
latent_sizes = [64, 128, 256]

# Training triplet loss

In [47]:
for part in parts:    
    train_set = EmbeddingDataset(image_path=image_pkl_path, text_path=text_pkl_path.replace('#replace#', part), part=part)
    val_set = EmbeddingDataset(image_path=image_pkl_path.replace('train', 'val'), text_path=text_pkl_path.replace('#replace#', part).replace('train', 'val'), part=part)
    train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_set, batch_size=batch_size, shuffle=False)

    for latent_size in latent_sizes:
        args.save_dir = f'notebook_latent_{latent_size}_{part}'
        ckpt_path = os.path.join(args.save_dir, 'model.pt')

        if os.path.exists(ckpt_path):
            print('Loading model')
            ckpt_args, epoch_start, txt_encoder, img_encoder, optimizer = load_retrieval_model(ckpt_path, device)
        else:
            print('Creating model')
            txt_encoder, img_encoder, optimizer = create_retrieval_model(args, device)

        if device == 'cuda':
            txt_encoder = nn.DataParallel(txt_encoder)
            img_encoder = nn.DataParallel(img_encoder)

        os.makedirs(args.save_dir, exist_ok=True)
        to_print = ['Now training', f'Latent size: {latent_size}', f'text part: {part}']
        print('\n'.join(to_print))
        train_triplet_loss(args, train_loader, val_loader, txt_encoder, img_encoder, optimizer, len(train_set), len(val_set))

    # Making space    
    for fpath in glob('./*.pkl'):
        os.remove(fpath)


Creating model
Now training
Latent size: 64
text part: all
########################################
Epoch = 0
train


100%|██████████| 275/275 [00:18<00:00, 14.95it/s]


val


100%|██████████| 60/60 [00:01<00:00, 39.11it/s]


=>retrieved_range=1000, MedR=1.0000(0.0000), time=3.4837s
save to notebook_latent_64_all/model.pt
{'epoch': 0, 'train_loss': 1.1231496344684613, 'val_loss': 0.774420966860878, 'medR': 1.0, 'medR_std': 0.0, 'Recall1': 0.579, 'Recall5': 0.8280000000000001, 'Recall10': 0.8897}
########################################
Epoch = 1
train


...

=>retrieved_range=1000, MedR=3.0000(0.0000), time=1.8030s
{'epoch': 28, 'train_loss': 0.5670856501574557, 'val_loss': 1.0516310526232375, 'medR': 3.0, 'medR_std': 0.0, 'Recall1': 0.343, 'Recall5': 0.6343, 'Recall10': 0.7308}
########################################
Epoch = 29
train


100%|██████████| 275/275 [00:19<00:00, 14.04it/s]


val


100%|██████████| 60/60 [00:01<00:00, 36.45it/s]


=>retrieved_range=1000, MedR=3.0000(0.0000), time=2.4205s
{'epoch': 29, 'train_loss': 0.5668979752409952, 'val_loss': 1.0494174796539553, 'medR': 3.0, 'medR_std': 0.0, 'Recall1': 0.34209999999999996, 'Recall5': 0.618, 'Recall10': 0.7216}


# Training MSE

In [50]:
for part in parts:    
    train_set = EmbeddingDataset(image_path=image_pkl_path, text_path=text_pkl_path.replace('#replace#', part), part=part)
    val_set = EmbeddingDataset(image_path=image_pkl_path.replace('train', 'val'), text_path=text_pkl_path.replace('#replace#', part).replace('train', 'val'), part=part)
    train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_set, batch_size=batch_size, shuffle=False)

    for latent_size in latent_sizes:
        args.save_dir = f'notebook_latent_{latent_size}_{part}_mse'
        ckpt_path = os.path.join(args.save_dir, 'model.pt')

        if os.path.exists(ckpt_path):
            print('Loading model')
            ckpt_args, epoch_start, txt_encoder, img_encoder, optimizer = load_retrieval_model(ckpt_path, device)
        else:
            print('Creating model')
            txt_encoder, img_encoder, optimizer = create_retrieval_model(args, device)

        if device == 'cuda':
            txt_encoder = nn.DataParallel(txt_encoder)
            img_encoder = nn.DataParallel(img_encoder)

        os.makedirs(args.save_dir, exist_ok=True)
        to_print = ['Now training', f'Latent size: {latent_size}', f'text part: {part}']
        print('\n'.join(to_print))
        train_mse(args, train_loader, val_loader, txt_encoder, img_encoder, optimizer, len(train_set), len(val_set))

    # Making space    
    for fpath in glob('./*.pkl'):
        os.remove(fpath)


Creating model
Now training
Latent size: 64
text part: all
########################################
Epoch = 0
train


100%|██████████| 275/275 [00:20<00:00, 13.38it/s]


val


100%|██████████| 60/60 [00:01<00:00, 32.91it/s]


=>retrieved_range=1000, MedR=1.0000(0.0000), time=2.2983s
save to notebook_latent_64_all_mse/model.pt
{'epoch': 0, 'train_loss': 1.1257691062328412, 'val_loss': 0.7800897790685596, 'medR': 1.0, 'medR_std': 0.0, 'Recall1': 0.5805, 'Recall5': 0.8309999999999998, 'Recall10': 0.8878999999999999}
########################################
Epoch = 1
train


...

=>retrieved_range=1000, MedR=2.9000(0.3000), time=2.2907s
{'epoch': 28, 'train_loss': 0.45342019932730754, 'val_loss': 0.7701872348758766, 'medR': 2.9, 'medR_std': 0.3, 'Recall1': 0.34839999999999993, 'Recall5': 0.6271, 'Recall10': 0.7252}
########################################
Epoch = 29
train


100%|██████████| 275/275 [00:20<00:00, 13.46it/s]


val


100%|██████████| 60/60 [00:01<00:00, 30.53it/s]


=>retrieved_range=1000, MedR=3.0000(0.0000), time=2.4455s
{'epoch': 29, 'train_loss': 0.4534770831389416, 'val_loss': 0.7701159917139485, 'medR': 3.0, 'medR_std': 0.0, 'Recall1': 0.3318, 'Recall5': 0.625, 'Recall10': 0.7241}
